# Data acquisition in Julia

It's time to convert the MATLAB code I wrote to pull RTS-96 data out of Jenny's Excel spreadsheets into Julia code.

I actually don't think I have enough time for this right now. Perhaps for the upcoming journal article.

Okay, I changed my mind. It's more important now to connect various parts of existing RTS-96 data to the temporal instanton solver.

In [ ]:
include("../src/TemporalInstanton.jl")
include("../src/TrustRegionSubproblem.jl")

using TemporalInstanton, TrustRegionSubproblem

In [1]:
include("../src/tmp_inst_rts96.jl")

psData = psDataLoad()

# unpack psDL (boilerplate):
(Sb,f,t,r,x,b,Y,bustype,
Gp,Gq,Dp,Dq,Rp,Rq,
Pmax,Pmin,Qmax,Qmin,Plim,
Vg,Vceiling,Vfloor,
busIdx,N,Nr,Ng,k) = unpack_psDL(psData)

(100.0,[1,1,1,2,2,3,3,4,5,6  …  66,66,67,67,68,68,69,73,66,71],[2,3,5,4,6,9,24,9,10,10  …  69,69,68,68,71,71,70,21,47,73],[0.003,0.055,0.022,0.033,0.05,0.031,0.002,0.027,0.023,0.014  …  0.003,0.003,0.005,0.005,0.003,0.003,0.009,0.012,0.013,0.0],[0.014,0.211,0.085,0.127,0.192,0.119,0.084,0.104,0.088,0.061  …  0.026,0.026,0.04,0.04,0.022,0.022,0.068,0.097,0.104,0.009],[0.461,0.057,0.023,0.034,0.052,0.032,0.0,0.028,0.024,2.459  …  0.055,0.055,0.083,0.083,0.046,0.046,0.142,0.203,0.218,0.0],
73x73 sparse matrix with 289 Float64 entries:
	[1 ,  1]  =  87.9326
	[2 ,  1]  =  -71.4286
	[3 ,  1]  =  -4.73934
	[5 ,  1]  =  -11.7647
	[1 ,  2]  =  -71.4286
	[2 ,  2]  =  84.5109
	[4 ,  2]  =  -7.87402
	[6 ,  2]  =  -5.20833
	[1 ,  3]  =  -4.73934
	[3 ,  3]  =  25.0475
	⋮
	[60, 71]  =  -10.3093
	[61, 71]  =  -11.4943
	[68, 71]  =  -90.9091
	[71, 71]  =  223.824
	[73, 71]  =  -111.111
	[51, 72]  =  -11.9048
	[63, 72]  =  -19.2308
	[72, 72]  =  31.1355
	[21, 73]  =  -10.3093
	[71, 73]  =  -111.111
	[73

In [30]:
####### DATA ENTRY ########

# Static
Ridx = find(Rp) # Vector of renewable nodes
Y = Y # Full admittance matrix (slack not removed)
slack = 1 # Index of slack node
k = k # Conventional generator participation factors
tau = 2 # Heating constant
G0 = Gp
D = Dp
P0 = Rp
kQobj = 0 # Should always be zero; objective has no constant term
c = 10 # RHS of line temperature constraint
kQtheta = -c # Move to LHS so we have Q(z) = 0
lines = [(f[i],t[i]) for i in 1:length(f)]

line = lines[1]

(1,2)

In [28]:
# Changing
for line in lines
    display(line)
end

(1,2)

(1,3)

(1,5)

(2,4)

(2,6)

...

(68,71)

(69,70)

(73,21)

(66,47)

(71,73)

In [ ]:
# Create instance of instanton problem:
Qobj,A,b,Qtheta = temporalInstanton(Ridx,Y,slack,k,tau,line,G0,D)

G_of_x = (Qobj,0,kQobj)
Q_of_x = (Qtheta,0,kQtheta)

function tr_trans_rotate(G_of_x,Q_of_x,A,b)
    H_of_x,R_of_x,x_star = tr_translate(G_of_x,Q_of_x,A,b)
    # R_of_x is rotation invariant.
    J_of_z,Rkernel = tr_kernel_rotate(H_of_x,A)
    K_of_w,Reigvec = tr_diag_rotate(J_of_z)
    
    return K_of_w,R_of_x,x_star,Rkernel,Reigvec
end

tr_trans_rotate(G_of_x,Q_of_x,A,b)

In [18]:
[[f[i],t[i]] for i in 1:10]

10-element Array{Any,1}:
 [1,2] 
 [1,3] 
 [1,5] 
 [2,4] 
 [2,6] 
 [3,9] 
 [3,24]
 [4,9] 
 [5,10]
 [6,10]

In [4]:
whos()

Base                          Module
Blosc                         Module
Compat                        Module
Core                          Module
DataStructures                Module
Dp                            73-element Array{Float64,1}
Dq                            73-element Array{Float64,1}
Gp                            73-element Array{Float64,1}
Gq                            73-element Array{Float64,1}
HDF5                          Module
IJulia                        Module
IPythonDisplay                Module
JLD                           Module
JSON                          Module
MAT                           Module
MAT_HDF5                      Module
MAT_v5                        Module
Main                          Module
N                             Int64
Nettle                        Module
Ng                            Int64
Nr                            Int64
Plim                          120-element Array{Float64,1}
Pmax                          99-element Array

In [7]:
sum(Rp.!=0)

18

In [9]:
find(Rp)

18-element Array{Int64,1}:
  2
 14
 16
 17
 18
 19
 20
 21
 23
 26
 36
 37
 43
 44
 47
 49
 54
 57

In [11]:
k

73-element Array{Float64,1}:
 0.030303
 0.030303
 0.0     
 0.0     
 0.0     
 0.0     
 0.030303
 0.030303
 0.0     
 0.0     
 0.0     
 0.0     
 0.030303
 ⋮       
 0.0     
 0.030303
 0.030303
 0.0     
 0.030303
 0.0     
 0.0     
 0.030303
 0.030303
 0.030303
 0.0     
 0.0     